<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/module1-logistic-regression/LS_DS_231_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

In [0]:
!pip install kaggle

In [0]:
# Note - you'll also have to sign up for Kaggle and authorize the API
# https://github.com/Kaggle/kaggle-api#api-credentials

# This essentially means uploading a kaggle.json file
# For Colab we can have it in Google Drive
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

# You also have to join the Titanic competition to have access to the data
!kaggle competitions download -c titanic

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 23.7MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.8MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.15MB/s]


In [0]:
# How would we try to do this with linear regression?
import pandas as pd

train_df = pd.read_csv('train.csv').dropna()
test_df = pd.read_csv('test.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [0]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.674426,0.464481,0.475410,78.682469
std,247.052476,0.470725,0.515187,15.643866,0.644159,0.754617,76.347843
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,36.000000,0.000000,0.000000,57.000000
75%,676.000000,1.000000,1.000000,47.500000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


In [0]:
from sklearn.linear_model import LinearRegression

X = train_df[['Pclass', 'Age', 'Fare']]
y = train_df.Survived

linear_reg = LinearRegression().fit(X, y)
linear_reg.score(X, y)

0.08389810726550917

In [0]:
linear_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([0.79543117, 0.58610823, 0.67595121, 0.793829  , 0.62090522,
       0.72542107, 0.59848968, 0.58734245, 0.48567063, 0.77627736,
       0.84211887, 0.57052439, 0.7754689 , 0.96621114, 0.70287941,
       0.57673837, 0.72321391, 0.75894755, 0.77968041, 0.50246003,
       0.49858077, 0.7474959 , 0.3542282 , 0.61648435, 0.71300224,
       0.66294608, 0.53175333, 0.77397395, 0.68419387, 0.68395536,
       0.52041202, 0.56814038, 0.79586606, 0.81372012, 0.61068545,
       0.57260627, 0.52525981, 0.58055388, 0.45584728, 0.67976208,
       0.8226707 , 0.84286197, 0.96189157, 0.66724612, 0.68589478,
       0.61846513, 0.63455044, 0.68275686, 0.65738372, 0.45198998,
       0.59988596, 0.63845908, 0.63132487, 0.7888473 , 0.60126246,
       0.79714045, 0.78713803, 0.54643775, 0.42823635, 0.7711724 ,
       0.53552976, 0.55608044, 0.54480459, 0.57031915, 0.65080369,
       0.77958926, 0.6371013 , 0.70993488, 0.71493598, 0.60375943,
       0.54407206, 0.48186138, 0.76576089, 0.75456305, 0.53968

In [0]:
linear_reg.coef_

array([-0.08596295, -0.00829314,  0.00048775])

In [0]:
import numpy as np

test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

array([1.14845883])

In [0]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7103825136612022

In [0]:
log_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [0]:
log_reg.predict(test_case)[0]

1

In [0]:
help(log_reg.predict)

Help on method predict in module sklearn.linear_model.base:

predict(X) method of sklearn.linear_model.logistic.LogisticRegression instance
    Predict class labels for samples in X.
    
    Parameters
    ----------
    X : array_like or sparse matrix, shape (n_samples, n_features)
        Samples.
    
    Returns
    -------
    C : array, shape [n_samples]
        Predicted class label per sample.



In [0]:
log_reg.predict_proba(test_case)[0]

array([0.02485552, 0.97514448])

In [0]:
# What's the math?
log_reg.coef_

array([[-0.0455017 , -0.02912513,  0.0048037 ]])

In [0]:
log_reg.intercept_

array([1.45878264])

In [0]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
  return 1 / (1 + np.e**(-x))

In [0]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

array([[0.97514448]])

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

For live - let's tackle [another classification dataset on absenteeism](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) - it has 21 classes, but remember, scikit-learn LogisticRegression automatically handles more than two classes. How? By essentially treating each label as different (1) from some base class (0).

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [2]:
# Live - let's try absenteeism!
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip
!unzip Absenteeism_at_work_AAA.zip	

--2019-01-27 06:57:55--  http://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66136 (65K) [application/zip]
Saving to: ‘Absenteeism_at_work_AAA.zip’

Absenteeism_at_work 100%[===================>]  64.59K   179KB/s    in 0.4s    

2019-01-27 06:58:01 (179 KB/s) - ‘Absenteeism_at_work_AAA.zip’ saved [66136/66136]

Archive:  Absenteeism_at_work_AAA.zip
  inflating: Absenteeism_at_work.arff  
  inflating: Absenteeism_at_work.csv  
  inflating: Absenteeism_at_work.xls  
  inflating: Attribute Information.docx  
  inflating: UCI_ABS_TEXT.docx       


In [0]:
!head Absenteeism_at_work.csv

ID;Reason for absence;Month of absence;Day of the week;Seasons;Transportation expense;Distance from Residence to Work;Service time;Age;Work load Average/day ;Hit target;Disciplinary failure;Education;Son;Social drinker;Social smoker;Pet;Weight;Height;Body mass index;Absenteeism time in hours
11;26;7;3;1;289;36;13;33;239.554;97;0;1;2;1;0;1;90;172;30;4
36;0;7;3;1;118;13;18;50;239.554;97;1;1;1;1;0;0;98;178;31;0
3;23;7;4;1;179;51;18;38;239.554;97;0;1;0;1;0;0;89;170;31;2
7;7;7;5;1;279;5;14;39;239.554;97;0;1;2;1;1;0;68;168;24;4
11;23;7;5;1;289;36;13;33;239.554;97;0;1;2;1;0;1;90;172;30;2
3;23;7;6;1;179;51;18;38;239.554;97;0;1;0;1;0;0;89;170;31;2
10;22;7;6;1;361;52;3;28;239.554;97;0;1;1;1;0;4;80;172;27;8
20;23;7;6;1;260;50;11;36;239.554;97;0;1;4;1;0;0;65;168;23;4
14;19;7;2;1;155;12;14;34;239.554;97;0;1;2;1;0;0;95;196;25;40


In [3]:
absent_df = pd.read_table('Absenteeism_at_work.csv', sep=';')
absent_df.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2


In [4]:
absent_df.shape

(740, 21)

In [0]:
absent_df.columns

Index(['ID', 'Reason for absence', 'Month of absence', 'Day of the week',
       'Seasons', 'Transportation expense', 'Distance from Residence to Work',
       'Service time', 'Age', 'Work load Average/day ', 'Hit target',
       'Disciplinary failure', 'Education', 'Son', 'Social drinker',
       'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index',
       'Absenteeism time in hours'],
      dtype='object')

In [12]:
from sklearn.preprocessing import minmax_scale
X = absent_df.drop('Reason for absence', axis='columns')
X = minmax_scale(X)

y = absent_df['Reason for absence']

absent_log1 = LogisticRegression().fit(X, y)
absent_log1.score(X, y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.43783783783783786

In [13]:
absent_log1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [17]:
X[0]

array([0.28571429, 0.58333333, 0.25      , 0.        , 0.63333333,
       0.65957447, 0.42857143, 0.19354839, 0.19447062, 0.84210526,
       0.        , 0.        , 0.5       , 1.        , 0.        ,
       0.125     , 0.65384615, 0.27272727, 0.57894737, 0.03333333])

In [18]:
absent_df.head(1)

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4


In [16]:
absent_log1.coef_[0]

array([-5.59083076e-01, -1.17869726e+00, -1.77728416e-01,  2.75874361e-01,
       -2.60616286e-01, -7.09081683e-01, -6.27006760e-01,  4.35004585e-01,
       -4.56034397e-01, -1.37206545e+00,  5.82849403e+00, -5.69515028e-01,
        2.41759658e-01, -4.33436812e-03,  1.64470286e-01,  7.51137725e-01,
        6.06027505e-02, -3.33297763e-01,  1.24032523e-02, -5.93146583e-01])

### For X[0], Pet(7.5), Weight(6.06) and Disciplinary_failure(5.8) have the highest correlation for the Reason_of_Absence=26.

In [15]:
absent_log1.predict_proba(X)[0]

array([0.01301273, 0.01644256, 0.00210923, 0.00345691, 0.00298031,
       0.00599653, 0.0092346 , 0.01515905, 0.00469207, 0.00459713,
       0.02267775, 0.02980498, 0.00649426, 0.0459122 , 0.02071804,
       0.00589661, 0.001923  , 0.00196416, 0.03427272, 0.08915994,
       0.0055056 , 0.11091894, 0.23630945, 0.01042832, 0.03749956,
       0.13308167, 0.00761263, 0.12213905])

### Reason of Absence 26 has 0.23 probability over the rest lower probability.

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

In [1]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-01-28 06:17:24--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip’

fma_metadata.zip    100%[===================>] 341.81M  18.3MB/s    in 42s     

2019-01-28 06:18:08 (8.05 MB/s) - ‘fma_metadata.zip’ saved [358412441/358412441]

Archive:  fma_metadata.zip
 bunzipping: fma_metadata/README.txt  
 bunzipping: fma_metadata/checksums  
 bunzipping: fma_metadata/not_found.pickle  
 bunzipping: fma_metadata/raw_genres.csv  
 bunzipping: fma_metadata/raw_albums.csv  
 bunzipping: fma_metadata/raw_artists.csv  
 bunzipping: fma_metadata/raw_tracks.csv  
 bunzipping: fma_metadata/tracks.csv  
 bunzipping: fma_metadata/genres.csv  
 bunzipping: fma_metadata/raw_echonest.cs

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [3]:
tracks = pd.read_csv('fma_metadata/tracks.csv')
tracks.head(3)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,...,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food


In [0]:
pd.set_option('display.max_columns', None)  # Unlimited columns
pd.options.mode.use_inf_as_na = True

In [4]:
# fixing double header
tracks = pd.read_csv('fma_metadata/tracks.csv', header=[0,1], index_col=0)
tracks.head(3)

album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
2               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
5               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   

                                                             ...       \
         favorites id information listens producer tags      ...        
track_id                                                     ...        
2                4  1     <p></p>    6073      NaN   []      ...        
3                4  1     <p></p>    6073      NaN   []      ...        
5                4  1     <p></p>    6073      NaN   []      ...        

               track                         \
         information interest language_code   
track_id                                      
2                NaN     4656            en   
3                NaN     1470            en   
5                NaN     1933            en   

                                                                              \
                                                    license listens lyricist   
track_id                                                                       
2         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1293      NaN   
3         Attribution-NonCommercial-ShareAlike 3.0 Inter...     514      NaN   
5         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1151      NaN   

                                              
         number publisher tags         title  
track_id                                      
2             3       NaN   []          Food  
3             4       NaN   []  Electric Ave  
5             6       NaN   []    This World  

[3 rows x 52 columns]

In [0]:
tracks.reset_index(drop=True, inplace=True)

In [6]:
tracks.head(1)

album                                                                  \
  comments         date_created        date_released engineer favorites id   
0        0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN         4  1   

                                     ...        track                         \
  information listens producer tags  ...  information interest language_code   
0     <p></p>    6073      NaN   []  ...          NaN     4656            en   

                                                                              \
                                             license listens lyricist number   
0  Attribution-NonCommercial-ShareAlike 3.0 Inter...    1293      NaN      3   

                        
  publisher tags title  
0       NaN   []  Food  

[1 rows x 52 columns]

In [7]:
tracks.shape

(106574, 52)

In [0]:
tracks=tracks.sample(2000, random_state=42)

In [9]:
tracks.describe()

album                                                       \
          comments    favorites           id       listens       tracks   
count  2000.000000  2000.000000   2000.00000  2.000000e+03  2000.000000   
mean      0.443500     1.285500  12850.28600  3.739742e+04    19.552500   
std       2.884262     3.086868   6219.58821  1.964804e+05    35.496877   
min      -1.000000    -1.000000     -1.00000 -1.000000e+00    -1.000000   
25%       0.000000     0.000000   7895.00000  3.309250e+03     7.000000   
50%       0.000000     0.000000  13370.00000  8.911500e+03    11.000000   
75%       0.000000     1.000000  18232.75000  2.463975e+04    18.000000   
max      53.000000    55.000000  22905.00000  3.564243e+06   652.000000   

            artist                                                     \
          comments    favorites            id    latitude   longitude   
count  2000.000000  2000.000000   2000.000000  836.000000  836.000000   
mean      2.218000    33.792500  12022.082000   38.662066  -34.336361   
std       7.181244   113.006777   6791.986517   20.769404   69.277985   
min      -1.000000    -1.000000      4.000000  -45.878760 -123.114034   
25%       0.000000     1.000000   6567.500000   39.243541  -80.289438   
50%       0.000000     5.000000  12038.500000   41.846417  -72.713299   
75%       1.000000    16.250000  17690.000000   48.938791    5.369780   
max      79.000000   951.000000  24296.000000   64.963051  174.885971   

               track                                                       \
            bit_rate     comments     duration    favorites      interest   
count    2000.000000  2000.000000  2000.000000  2000.000000  2.000000e+03   
mean   265349.416500     0.037000   282.718000     3.144000  4.299708e+03   
std     66587.921464     0.371079   295.134104    16.531119  3.374043e+04   
min        -1.000000     0.000000     1.000000     0.000000  7.000000e+00   
25%    196496.000000     0.000000   152.000000     0.000000  5.847500e+02   
50%    320000.000000     0.000000   220.000000     1.000000  1.247500e+03   
75%    320000.000000     0.000000   308.000000     3.000000  3.099250e+03   
max    320817.000000    11.000000  3710.000000   599.000000  1.314156e+06   

                                   
             listens       number  
count    2000.000000  2000.000000  
mean     2567.795000     8.137000  
std     12456.774413    13.616899  
min         3.000000     0.000000  
25%       264.000000     2.000000  
50%       729.000000     5.000000  
75%      2031.250000     9.000000  
max    429168.000000   206.000000

In [0]:
# Wrong approach. Use DataFrame._get_numeric_data() to scoop up all numeric features.
X=pd.DataFrame()
X['album_favorites']=pd.to_numeric(tracks.album.favorites)
X['album_comments']=tracks.album.comments
X['album_listens']=tracks.album.listens
X['album_tracks']=tracks.album.tracks
X['artist_favorites']=tracks.artist.favorites
X['track_favorites']=tracks.track.favorites

X['track_interest']=tracks.track.interest
X['track_listens']=tracks.track.listens

In [10]:
X= tracks._get_numeric_data()
X.shape

(2000, 17)

In [11]:
X.isna().sum()

album   comments        0
        favorites       0
        id              0
        listens         0
        tracks          0
artist  comments        0
        favorites       0
        id              0
        latitude     1164
        longitude    1164
track   bit_rate        0
        comments        0
        duration        0
        favorites       0
        interest        0
        listens         0
        number          0
dtype: int64

In [0]:
X=X.replace(np.nan,0)

In [12]:
X.dtypes

album   comments       int64
        favorites      int64
        id             int64
        listens        int64
        tracks         int64
artist  comments       int64
        favorites      int64
        id             int64
        latitude     float64
        longitude    float64
track   bit_rate       int64
        comments       int64
        duration       int64
        favorites      int64
        interest       int64
        listens        int64
        number         int64
dtype: object

In [13]:
X.head()

album                                   artist                   \
       comments favorites     id listens tracks comments favorites     id   
28944        -1        -1   8578      -1     -1        4        82   8390   
20194         0         1   6655   19141     14        1        15   7888   
33505         3        10   9619  458874     20       32       431  11431   
100878        0         0  21917   19158     11        4       110  15891   
12580         0         1   4611    3139      5        1         6   5443   

                                track                                       \
         latitude  longitude bit_rate comments duration favorites interest   
28944   39.263517 -76.623942   160000        0      325         2     2629   
20194   46.211401   2.209360   320000        0      168         2     2105   
33505   40.692455 -73.990364   267146        0      174        26    31224   
100878  58.007359  56.228149   320000        0      308         0     3491   
12580   41.871940  12.567380   171259        0       86         0      722   

                       
       listens number  
28944      850      8  
20194     1320      9  
33505    19783      0  
100878    2908     10  
12580      385      6

In [0]:
y=tracks.track.genre_top

In [15]:
y.unique()

array([nan, 'Hip-Hop', 'Rock', 'Electronic', 'Experimental', 'Folk',
       'Pop', 'International', 'Classical', 'Instrumental', 'Jazz',
       'Old-Time / Historic', 'Easy Listening', 'Spoken', 'Soul-RnB'],
      dtype=object)

In [0]:
y=y.replace(np.nan, 'others')


In [17]:
y.unique()

array([nan, 'Hip-Hop', 'Rock', 'Electronic', 'Experimental', 'Folk',
       'Pop', 'International', 'Classical', 'Instrumental', 'Jazz',
       'Old-Time / Historic', 'Easy Listening', 'Spoken', 'Soul-RnB'],
      dtype=object)

In [21]:
#Scale X for better Logistic Regression

from sklearn.preprocessing import minmax_scale

X= tracks._get_numeric_data()
X=X.replace(np.nan,0)
colnames=X.columns

X=minmax_scale(X)

y=tracks.track.genre_top
y=y.replace(np.nan,'others')


model = LogisticRegression().fit(X,y)
log_score=model.score(X,y)

zipped = zip(colnames, model.coef_[0])

print(f'\nLogReg score={log_score}')
#print(f'Genre Top={y[28944]}')
print('\nLogReg coefficient (using minmax_scale between [0,1])')
for col, coef in sorted(zipped, key=lambda t:t[1], reverse=True):
  print(f'{col}:\t\t{coef}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64.
  
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)



LogReg score=0.5515

LogReg coefficient (using minmax_scale between [0,1])
('album', 'favorites'):		0.6650113939581133
('artist', 'favorites'):		0.5508932388895665
('track', 'duration'):		0.3910929911900119
('artist', 'comments'):		0.3840233240636232
('album', 'listens'):		0.2007106404397927
('track', 'favorites'):		0.1396952006599037
('artist', 'id'):		0.13457180454574477
('track', 'listens'):		0.11895168277871414
('track', 'number'):		0.05636747209907498
('album', 'tracks'):		0.04583468581373385
('track', 'interest'):		0.03749627611541833
('album', 'comments'):		-0.16854611500743474
('track', 'comments'):		-0.17471691111702306
('album', 'id'):		-0.47710720781184796
('artist', 'latitude'):		-0.5323273520031039
('artist', 'longitude'):		-0.7154469860885448
('track', 'bit_rate'):		-2.3110056841035336


In [137]:

X= tracks._get_numeric_data()
X=X.replace(np.nan,0)
colnames=X.columns

X=minmax_scale(X)

y=tracks.track.genre_top
y=y.replace(np.nan,'others')


genre_top_logReg = LogisticRegression().fit(X,y)
log_score=genre_top_logReg.score(X,y)

zipped = zip(colnames, genre_top_logReg.coef_[1])

print(f'\nLogReg score={log_score}')
print(f'Genre Top={y[20194]}')
print('\nLogReg coefficient (using minmax_scale between [0,1])')
for col, coef in sorted(zipped, key=lambda t:t[1], reverse=True):
  print(f'{col}:\t\t{coef}')


LogReg score=0.5515
Genre Top=others

LogReg coefficient (using minmax_scale between [0,1])
('artist', 'favorites'):		0.32099698775559143
('artist', 'comments'):		0.25809295308100505
('album', 'listens'):		0.043361276177399065
('track', 'listens'):		0.03916581899669565
('track', 'interest'):		0.013472261194876739
('track', 'favorites'):		0.007260052205069408
('track', 'comments'):		-0.01692377662993002
('album', 'favorites'):		-0.05136516482336137
('album', 'comments'):		-0.08405663960724344
('album', 'tracks'):		-0.0972931232522939
('track', 'number'):		-0.1369187505866062
('track', 'duration'):		-0.21070135342380367
('artist', 'id'):		-0.8822588371513019
('artist', 'longitude'):		-1.130926902428303
('album', 'id'):		-1.131622856952524
('artist', 'latitude'):		-1.358339580865497
('track', 'bit_rate'):		-1.7130860208079168


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64.
  
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.